### Implement a deep neural network, which contains two hidden layers (the hidden layers are obtained from the ELM-autoencoders). The last layer will be the ELM layer which means the second hidden layer feature vector is used as input to the ELM classifier. The network can be called as deep layer stacked autoencoder based extreme learning machine. You can use holdout approach (70, 10, 20%) for evaluating the performance of the classifier. The dataset (data5.mat) contains 72 features and the last column is the output (class labels). Evaluate individual accuracy and overall accuracy. (Packages such as keras, tensorflow, pytorch for python and MATLAB deep learning toolbox etc. are not allowed)


In [1]:
#import all the necessory libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from mpl_toolkits import mplot3d
import scipy.io

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
root="/content/drive/MyDrive/NNFL_Assignment2/data5.mat"
data=scipy.io.loadmat(os.path.join(root))
data=data['x']
print((data.shape))


(2148, 73)


In [4]:
def onehoten(data):
    y=data[:,-1]
    count=0
    cls=[]
    for insts in range(0,y.shape[0]):
        if y[insts] in cls:
            continue
        else:
            cls.append(y[insts])
    nclasses=len(cls)
    z=np.zeros([y.shape[0],nclasses],dtype=int)
    for insts in range(0,y.shape[0]):
        z[insts,int(y[insts])]=z[insts,int(y[insts])]+1
    return z
        

In [5]:
def split_classes(data):
    x=np.delete(data,-1,1)
    y=onehoten(data)
    c=[]
    for cls in range(0,y.shape[1]):
        temp=[]
        for insts in range(data.shape[0]):
            if y[insts,cls]==1:
                temp.append(data[insts])
        c.append(np.array(temp))
    return(c)

In [20]:
def split_train_test(classes,trainperc):
    for c in range(len(classes)):
        dat=classes[c]
        train=dat[:round(dat.shape[0]*trainperc/100),:]
        test=dat[round(dat.shape[0]*trainperc/100):,:]
        if c!=0:
            train=np.concatenate((trainold,train),axis=0)
            test=np.concatenate((testold,test),axis=0)
        trainold=train
        testold=test
    np.random.shuffle(train)
    np.random.shuffle(test)
        
    return train , test


In [7]:
def split_x_y(data):
    x=np.delete(data,-1,1)
    x=normalize(x)
#   x=np.concatenate((x,np.ones((data.shape[0],1))),axis=1)
    y=onehoten(data)
    # y=x
    return x,y

In [8]:
def normalize(data):
    for i in range(data.shape[1]):
        temp=data[:,i]
        data[:,i]=(temp-np.mean(temp))/np.std(temp)
    return data

In [10]:
def forward_prop(w,x_train):
    i=x_train
    for l in range(len(w)):
        i=np.concatenate((i,np.ones((i.shape[0],1))),axis=1)
        h=np.dot(i,w[l])
        i=h
    return i


In [11]:
def tanh(x,a):
    act=(1-np.exp(-1*(np.dot(x,a))))/(1+np.exp(-1*(np.dot(x,a))))
    return act

In [103]:
def pretrain(l,x_train,rate):
    w=[[] for j in range(len(l)-2)]
    inp=x_train
    for i in range(len(l)-2):
        inp=np.concatenate((inp,np.ones((inp.shape[0],1))),axis=1)
        we=np.random.rand(l[i]+1,l[i+1])
        h=tanh(inp,we)
        w2=np.dot(np.linalg.inv(np.dot(h.T,h)+(np.identity(h.shape[1])/rate)),np.dot(h.T,inp))
        w[i]=w2.T
        g=np.dot(inp,w2.T)
        inp=g
    return w,h




In [13]:
def accur(pred,y):
    count=0
    for inst in range(y.shape[0]):
        if np.argmax(pred[inst])==np.argmax(y[inst]):
            count=count+1
    acc=count/y.shape[0]
    return acc

In [146]:
def ELMENC(data,percepset,rate):
    train,test=split_train_test(split_classes(data),80)
    x_train,y_train=split_x_y(train)
    x_test,y_test=split_x_y(test)
    x_test=np.concatenate((x_train,x_test),axis=0)
    y_test=np.concatenate((y_train,y_test),axis=0)
    w,hf=pretrain(percepset,x_train,rate)
    hf=np.concatenate((hf,np.ones((hf.shape[0],1))),axis=1)
    wf=np.dot(np.linalg.inv(np.dot(hf.T,hf)+(np.identity(hf.shape[1])/rate)),np.dot(hf.T,y_train))
    w.append(wf)
    pred=np.dot(hf,wf)
    predtes=forward_prop(w,x_test)
    predtest=np.zeros((predtes.shape[0],1))
    for p in range(predtes.shape[0]):
        predtest[p]=np.argmax(predtes[p])

    h_test=(predtest).reshape((y_test.shape[0],1))
    y_test=y_test[:,1].reshape((y_test.shape[0],1))
    error = (y_test-np.rint(h_test)).tolist()
    som = (y_test+np.rint(h_test)).tolist()
    FN = error.count([1.0])
    FP = error.count([-1.0])
    TN = som.count([0.0])
    TP = som.count([2.0])
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    sensitivity = TP/(TP+FN)
    specificity = TN/(FP+TN)
    conf_matrix = np.array(([TP,FP],[FN,TN]))

    return accuracy,sensitivity ,specificity, conf_matrix 

In [157]:
accs,sens,specs,confmat=ELMENC(data,[72,1000,1000,2],0.008)

In [158]:
print("Accuracy={}".format(accs))
print("Sensitivity={}".format(sens))
print("Specificity={}".format(specs))
print(confmat)

Accuracy=0.7779329608938548
Sensitivity=0.7781919850885368
Specificity=0.7776744186046511
[[835 239]
 [238 836]]
